In [ ]:
import numpy as np
from scipy.stats import multivariate_normal as mvn
import matplotlib.pylab as plt
import sys
sys.path.insert(0, '../zdrojaky/kf')
from kf import KF
from trajectory import trajectory
np.set_printoptions(precision=2)

# 2D object tracking with particle filter

Our task is the filtration of 2D moving object location. We already did this with the Kalman filter, now, we will use the particle filtering methods. 

Recall, that we have a state vector consisting of locations in axes $x_1$ and $x_2$, and velocities in these axes.

$$
x_t =
\begin{bmatrix}
x_{1,t} \\ 
x_{2,t} \\ 
v_{x_1,t} \\ 
v_{x_2,t}
\end{bmatrix}
$$

We know that the $x_1$ location is given by

$$
x_{1,t} = x_{1,t-1} + v_{x_1,t} dt + w_{x_1,t},
$$

and that the same holds for the second axis. We will assume that the velocity is virtually constant, and its changes are due to the state noise,

$$
v_{x_1,t} = v_{x_1, t-1} + w_{vx_1, t}.
$$

Analogy holds for the other axis. We acquire noisy location measurements, the time step is 0.1s.

The state-space model is

$$
\begin{aligned}
x_t &\sim \mathcal{N}(Ax_{t-1} + Bu_t, Q),\\
y_t &\sim \mathcal{N}(Hx_{t}, R),
\end{aligned}
$$

and the covariance matrices are provided below.

\begin{align*}
    A &=
    \begin{bmatrix}
       1 & 0 & dt & 0 \\
       0 & 1 & 0 & dt \\
       0 & 0 & 1 &  0 \\
       0 & 0 & 0 &  1 
    \end{bmatrix},
    \quad
    &Q &= q\cdot
    \begin{bmatrix}
        \frac{dt^3}{3}    & 0                 & \frac{dt^{2}}{2}  & 0  \\
        0                 & \frac{dt^3}{3}    & 0                 & \frac{dt^{2}}{2} \\
        \frac{dt^{2}}{2}  & 0                 & dt                & 0 \\
        0                 & \frac{dt^{2}}{2}  & 0                 & dt
    \end{bmatrix}
    \notag \\
    H &=
    \begin{bmatrix}
        1 & 0 &0 & 0 \\
        0 & 1 &0 & 0
    \end{bmatrix}
    \quad
    &R &=
    r^{2}\cdot
    \begin{bmatrix}
        1 & 0 \\
        0 & 1
    \end{bmatrix}
\end{align*}
where $dt = 1$, $q = .5$, $r=3$.

** Task: complete the variables**

In [ ]:
q = .5
dt = 1
r = 3.
A = np.array([[1, 0, dt, 0],
              [0, 1, 0, dt],
              [0, 0, 1,  0],
              [0, 0, 0,  1]])
Q = q * np.array([[dt**3/3, 0      , dt**2/2, 0      ],
                  [0,       dt**3/3, 0,       dt**2/2],
                  [dt**2/2, 0,       dt,      0      ],
                  [0,       dt**2/2, 0,       dt     ]])
H = np.array([[1., 0, 0, 0],
              [0., 1, 0, 0]])
R = r**2 * np.eye(2)

**Task: let `code` be the day and month of your birth. In the `traj` object you find the trajectory: 100 rows with locations in both axes. Plot it with crosses.**

In [ ]:
code = ***
traj = trajectory(code)
plt.figure(figsize=(10, 10))
plt.plot(traj.Y[0,:], traj.Y[1,:], '+')
plt.axis('equal')
plt.show()

**Task: Define the number of samples and the proposal distribution N(0, 10I). Set and normalize the weights vector.**

In [ ]:
ndat = traj.ndat
nsamples = ***
proposal_loc = ***
proposal_cov = ***

samples_x = mvn.rvs(mean=proposal_loc, cov=proposal_cov, size=nsamples)
weights = ***

print("Weigths check: 1 =", weights.sum())

**Task: complete where you see asterisks**

In [ ]:
# Logs
log_x = np.zeros((4, ndat))

def prediction(samples_x):
    samples_x_new = np.zeros_like(samples_x)
    for i in range(nsamples):
        mean = ***             # Mean of the predicted pdf
        samples_x_new[i] = *** # Correctly sample from mvn.rvs()
    return samples_x_new

def update(yt, samples_x, weights):
    for i in range(nsamples):
        mean = ***        # Calculate the expected yt given H and samples_x[i]
        likelihood = ***  # Determine the pdf value of yt for given mean and covariance R with mvn.pdf()
        weights[i] *= likelihood
    weights = *** # Normalize weights
    return weights

def resample(samples_x, weights):
    indices = np.random.choice(np.arange(weights.size), replace=***, p=***, size=weights.size)
    samples_x_new = samples_x[indices]
    weights = *** # Set weights after resampling - think twice! ;)
    return [samples_x_new, weights]

In [ ]:
for t, yt in enumerate(traj.Y.T):
    print('{0} '.format(t), end='')
    
    # Resampling
    samples_x, weights = resample(samples_x, weights)
    
    # Prediction
    samples_x = prediction(samples_x)

    # Update
    weights = update(yt, samples_x, weights)
    
    # Estimate & log
    estimate = np.sum(samples_x * weights[:,np.newaxis], axis=0)
    log_x[:,t] = estimate

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(traj.Y[0,:], traj.Y[1,:], '.', label='Measurements')
plt.plot(log_x[0,:], log_x[1,:], '-', color='red', label='Filtered estimate')
plt.plot(traj.X[0,:], traj.X[1,:], 'k', label='True trajectory')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.plot(log_x[2,:], 'r')
plt.plot(traj.X[2,:], 'k')
plt.subplot(1,2,2)
plt.plot(log_x[3,:])
plt.plot(traj.X[3,:], 'k')
plt.show()